# 🤖 XGBoost 模型训练

本笔记本展示如何训练 XGBoost 3分类模型进行交易信号预测。

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ml_framework import FeatureEngineer, MLSignalPredictor

## 1. 数据准备

In [ ]:
# 加载数据
data_path = Path.cwd().parent / 'data' / 'TOPIX_18months_clean.csv'
if data_path.exists():
    df = pd.read_csv(data_path, nrows=50000)
    df['datetime'] = pd.to_datetime(df['datetime'])
    print(f'✅ 加载 {len(df):,} 行数据')
else:
    dates = pd.date_range('2024-01-01', periods=5000, freq='1min')
    df = pd.DataFrame({
        'datetime': dates,
        'open': 2400 + np.cumsum(np.random.randn(5000) * 2),
        'high': 2400 + np.cumsum(np.random.randn(5000) * 2) + 5,
        'low': 2400 + np.cumsum(np.random.randn(5000) * 2) - 5,
        'close': 2400 + np.cumsum(np.random.randn(5000) * 2),
        'volume': np.random.randint(1000, 50000, 5000)
    })
    print('⚠️ 使用示例数据')

In [ ]:
# 创建特征
engineer = FeatureEngineer()
df = engineer.create_all_features(df)
df = df.dropna()
print(f'清理后: {len(df):,} 行')

## 2. 生成标签

In [ ]:
# 生成交易标签
lookahead = 20
profit_threshold = 0.003

df['future_high'] = df['high'].rolling(lookahead, min_periods=1).max().shift(-lookahead)
df['future_low'] = df['low'].rolling(lookahead, min_periods=1).min().shift(-lookahead)

upside = (df['future_high'] - df['close']) / df['close']
downside = (df['close'] - df['future_low']) / df['close']

df['label'] = 0  # Neutral
df.loc[upside >= profit_threshold, 'label'] = 1  # Long
df.loc[downside >= profit_threshold, 'label'] = -1  # Short

df = df.iloc[:-lookahead]

print('标签分布:')
print(df['label'].value_counts())

## 3. 训练模型

In [ ]:
# 划分训练/测试集
split_idx = int(len(df) * 0.8)
train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

exclude_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume', 'label', 'future_high', 'future_low']
feature_cols = [c for c in df.columns if c not in exclude_cols]

X_train = train_df[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
y_train = train_df['label'].values

X_test = test_df[feature_cols].fillna(0).replace([np.inf, -np.inf], 0)
y_test = test_df['label'].values

print(f'训练集: {len(X_train):,}')
print(f'测试集: {len(X_test):,}')

In [ ]:
# 训练 XGBoost 模型
model = MLSignalPredictor(n_estimators=200, max_depth=6, learning_rate=0.05)
model.train(X_train, y_train, verbose=True)

## 4. 特征重要性

In [ ]:
# 获取特征重要性
importance_df = model.get_feature_importance(top_n=20)

plt.figure(figsize=(12, 8))
plt.barh(range(len(importance_df)), importance_df['importance'])
plt.yticks(range(len(importance_df)), importance_df['feature'])
plt.xlabel('重要性', fontsize=12)
plt.title('Top 20 特征重要性', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. 预测和评估

In [ ]:
# 生成预测
signals = model.predict(X_test, confidence_threshold=0.20)

from sklearn.metrics import classification_report

print('分类报告:')
print(classification_report(y_test, signals, target_names=['Short', 'Neutral', 'Long']))

## 下一步

- **04_optimization.ipynb** - Walk-Forward 优化
- **05_performance_analysis.ipynb** - 性能分析